In [78]:
import psycopg2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
#import plotly
#import plotly.graph_objs as go

#import plotly.plotly as py
#import cufflinks as cf


#from sklearn.metrics import r2_score

import rpy2
print(rpy2.__version__)

#import warnings
#from collections import Counter

2.9.4


In [4]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

#import R's "forcast "

ts=robjects.r('ts')
#forec=importr('forecast')
#forcast = importr('forecast')


stats = importr('stats')


In [79]:

try:
    connection = psycopg2.connect(user="sesyr",
                                  password="SeSyR54",
                                  host="51.254.68.36",
                                  port="5432",
                                  database="donnees_trafic")
    cursor = connection.cursor()

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
    postgreSQL_select_Query = """
    WITH metadata AS (
          SELECT ist.macrosection,
         ist.code_centrale,
         cnl.code_canal,
         sc.sens,
         cc.id_nature_mesure, cc.libelle
        FROM rdv.info_section_troncon ist
        JOIN rdv.sens_canal sc ON sc.sens::text = ist.sens::text
    JOIN myrabel.centrale c ON c.code_centrale = ist.code_centrale::bpchar
    JOIN myrabel.canal cnl ON cnl.id_centrale = c.id_centrale AND cnl.code_canal::text = sc.canal::text
    JOIN myrabel.cal_canal cc ON cc.id_canal = cnl.id_canal AND cc.libelle ~~ '%QT_360 %'::text
    
    ), metadatavt AS (
          SELECT distinct ist.macrosection,
         ist.code_centrale,
         cnl.code_canal,
         sc.sens,
         cc.id_nature_mesure, cc.libelle
        FROM rdv.info_section_troncon ist
        JOIN rdv.sens_canal sc ON sc.sens::text = ist.sens::text
    JOIN myrabel.centrale c ON c.code_centrale = ist.code_centrale::bpchar
    JOIN myrabel.canal cnl ON cnl.id_centrale = c.id_centrale AND cnl.code_canal::text = sc.canal::text
    JOIN myrabel.cal_canal cc ON cc.id_canal = cnl.id_canal AND cc.libelle ~~ '%VT_360 %'::text
    )
    
    SELECT 
       
       m.macrosection,
       m.sens,
       m.code_centrale,
       m.code_canal, 
       
        
       ts,
       dq.valeur AS qt,
       dv.valeur AS vt
       
       
      FROM metadata  m
      JOIN metadatavt v
      ON m.macrosection = v.macrosection AND m.sens = v.sens AND m.code_centrale = v.code_centrale AND  m.code_canal=  v.code_canal
      
      CROSS JOIN generate_series('2017-05-01' , '2017-05-07','00:06:00'::interval) AS  ts(ts)
      
      LEFT JOIN myrabel.mesure_2 dq    
      ON dq.id_nature_mesure = m.id_nature_mesure
      AND dq.date_mesure = ts 
      
        LEFT JOIN myrabel.mesure_2 dv    
      ON dv.id_nature_mesure = v.id_nature_mesure
      AND dv.date_mesure = ts
      
      
        --WHERE(  dv.date_mesure IS NULL OR dq.date_mesure IS NULL OR dv.date_mesure = dq.date_mesure )
      
     -- WHERE dq.date_mesure IS NOT  NULL AND dv.date_mesure IS NULL  --OR dq.date_mesure IS NULL
      
      """
    

    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from mobile table using cursor.fetchall")
    records = cursor.fetchall() 
   
   # print("Print each row and it's columns values")
   #for row in records:
       #   print("Macrosection = ", row[0])
        #  print("sens = ", row[1])
         # print("code centrale   = ", row[2])
          #print(" code canal = ", row[3])
        #  print("date_mesure = ", row[4])
         # print("qt_360 = ", row[5]) 
          #print("Vt_360 = ", row[6],"\n" )
            
            
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
        
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")            

You are connected to -  ('PostgreSQL 10.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-28), 64-bit',) 

Selecting rows from mobile table using cursor.fetchall
PostgreSQL connection is closed


In [80]:
df = pd.DataFrame(records, columns=['Macrosection','sens','code_centrale','code_canal','date','qt_360','vt_360'])

#strip code canal which is a char of 3 character with whitespace

df['code_canal'] = df['code_canal'].str.strip()

# add a column that its default value is np.nan

# df.missing

#sorting dates ?
#df = df.sort_values('date')

# replace zeros also with NAN

df = df.replace(0,np.NaN) # np.NaN ca va marcher , np.nan doesnt work anymore !!

# Descriptive Statistics

df.head()

,Macrosection,sens,code_centrale,code_canal,date,qt_360,vt_360
0,I11,SENS_1,MZE54.3,0,2017-05-01 00:00:00+02:00,33.0,97.0
1,I11,SENS_1,MZE54.3,0,2017-05-01 00:06:00+02:00,26.0,95.0
2,I11,SENS_1,MZE54.3,0,2017-05-01 00:12:00+02:00,36.0,98.0
3,I11,SENS_1,MZE54.3,0,2017-05-01 00:18:00+02:00,29.0,96.0
4,I11,SENS_1,MZE54.3,0,2017-05-01 00:24:00+02:00,37.0,94.0


In [81]:
DataFrameDict=df.groupby(['code_canal','code_centrale'])
    
#print to s
for key, item in DataFrameDict:
    print(DataFrameDict.get_group(key).count(), "\n\n")
    #DataFrameDict.get_group(key).to_csv(r'C:\Users\hp\Desktop\Stage IOT TS\devellopement\grouping_by_codes'+ str(key) +'.csv')
    


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1441
vt_360           1441
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1441
vt_360           1441
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1441
vt_360           1441
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1440
vt_360           1440
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1440
vt_360           1440
dtype: int64 


Macrosection     1441
sens             1441
code_centrale    1441
code_canal       1441
date             1441
qt_360           1440
vt_360           1

In [82]:
#store data in alias dictionary

DataGp = {elem : pd.DataFrame for elem,it in DataFrameDict}

for key in DataGp.keys():
        print(key,end ="  ")
        DataGp[key] = DataFrameDict.get_group(key)
#27 groups        

('0', 'MZE54.3')  ('0', 'MZE54.H')  ('0', 'SITE_03')  ('0', 'SITE_05')  ('0', 'SITE_08')  ('1', 'MZE54.3')  ('1', 'MZE54.4')  ('1', 'MZE54.H')  ('1', 'MZE54.K')  ('1', 'SITE_01')  ('1', 'SITE_03')  ('1', 'SITE_05')  ('1', 'SITE_08')  ('1', 'SITE_09')  ('2', 'MZE54.H')  ('2', 'SITE_03')  ('2', 'SITE_05')  ('2', 'SITE_08')  ('3', 'MZE54.H')  ('3', 'SITE_01')  ('3', 'SITE_03')  ('3', 'SITE_05')  ('3', 'SITE_08')  ('3', 'SITE_09')  ('5', 'SITE_01')  ('5', 'SITE_08')  ('5', 'SITE_09')  

In [83]:

from IPython.display import display

pd.options.display.max_columns = None
#numerical attributes:
display(DataFrameDict.describe(percentiles=[.50,.25,.75 ,.10, .90]))


qt_360                                            \
                           count        mean         std  min   10%    25%   
code_canal code_centrale                                                     
0          MZE54.3        1441.0  131.997224  101.243642  3.0  19.0  35.00   
           MZE54.H        1441.0  131.937543  104.571196  2.0  18.0  32.00   
           SITE_03        1441.0   74.525330   42.641524  2.0  18.0  30.00   
           SITE_05        1440.0   81.536111   50.428037  1.0  18.0  30.75   
           SITE_08        1440.0   85.266667   53.655696  2.0  18.0  30.00   
1          MZE54.3        1440.0  138.448611  104.295013  1.0  17.9  35.75   
           MZE54.4        1441.0  114.712006   91.762312  1.0  14.0  29.00   
           MZE54.H        1441.0  114.734906   84.507736  1.0  16.0  31.00   
           MZE54.K        1441.0  115.067314   91.117928  1.0  14.0  28.00   
           SITE_01        1434.0   36.077406   22.172867  1.0   6.0  13.00   
           SITE_03        1440.0   80.993750   47.843422  1.0  17.0  32.00   
           SITE_05        1439.0   82.904100   49.321395  1.0  17.0  32.00   
           SITE_08        1393.0   11.485284    6.959445  1.0   3.0   6.00   
           SITE_09        1407.0   24.641791   16.507627  1.0   5.0  10.00   
2          MZE54.H        1115.0    5.084305    3.804712  1.0   1.0   2.00   
           SITE_03        1329.0   62.112114   63.156315  1.0   2.0   7.00   
           SITE_05        1352.0   53.303994   54.068890  1.0   2.0   6.00   
           SITE_08        1279.0   52.719312   55.819967  1.0   2.0   5.00   
3          MZE54.H        1311.0   25.618612   28.699247  1.0   2.0   5.00   
           SITE_01        1228.0   28.249186   27.400821  1.0   1.0   4.00   
           SITE_03        1308.0   62.609327   60.093547  1.0   2.0   7.00   
           SITE_05        1326.0   59.076923   59.223696  1.0   2.0   6.00   
           SITE_08        1439.0   76.037526   52.632917  1.0  10.0  23.00   
           SITE_09        1441.0   66.545455   42.590192  1.0  12.0  25.00   
5          SITE_01        1435.0   60.134495   44.403163  1.0   9.0  18.00   
           SITE_08        1416.0   48.093927   47.870633  1.0   3.0   6.00   
           SITE_09        1314.0   56.412481   54.065307  1.0   2.0   7.00   

                                                      vt_360              \
                            50%    75%    90%    max   count        mean   
code_canal code_centrale                                                   
0          MZE54.3        119.0  203.0  273.0  400.0  1441.0   96.670368   
           MZE54.H        119.0  202.0  270.0  450.0  1441.0   90.819570   
           SITE_03         81.0  113.0  126.0  158.0  1441.0   94.820191   
           SITE_05         85.0  124.0  143.0  209.0  1440.0   94.586821   
           SITE_08         90.0  132.0  149.1  205.0  1440.0   84.723101   
1          MZE54.3        143.0  211.0  283.0  414.0  1440.0   96.886806   
           MZE54.4        116.0  173.0  213.0  461.0  1441.0  103.768217   
           MZE54.H        117.0  177.0  227.0  358.0  1441.0   98.566273   
           MZE54.K        114.0  176.0  219.0  433.0  1441.0   98.297016   
           SITE_01         42.0   55.0   63.0   85.0  1435.0  101.315459   
           SITE_03         97.0  121.0  139.1  175.0  1440.0   97.071148   
           SITE_05         98.0  126.0  141.0  184.0  1439.0  100.883613   
           SITE_08         10.0   17.0   21.0   49.0     0.0         NaN   
           SITE_09         21.0   40.0   47.0   70.0  1406.0   79.502892   
2          MZE54.H          4.0    7.0   10.0   20.0  1114.0   59.887792   
           SITE_03         47.0   93.0  158.2  281.0  1306.0  107.896635   
           SITE_05         40.0   80.0  133.9  228.0  1344.0  108.659105   
           SITE_08         41.0   77.0  131.0  246.0  1259.0   93.042972   
3          MZE54.H         18.0   35.0   50.0  171.0  1309.0   53.873950   
           SI

In [74]:
# Create table for missing data analysis
def find_missing_data(data):
    Total = data[['qt_360','vt_360']].isnull().sum()#.sort_values(ascending = False)
    Percentage = (data[['qt_360','vt_360']].isnull().sum()/data[['qt_360','vt_360']].isnull().count())#.sort_values(ascending = False)
    
    return pd.concat([Total,Percentage] , axis = 1 , keys = ['Total' , 'Percent'])

In [75]:
for key, item  in DataFrameDict:
         print("the key is" +str(key)+'\n', find_missing_data(DataFrameDict.get_group(key)),"\n \n")

the key is('0', 'MZE54.3')
         Total  Percent
qt_360      0      0.0
vt_360      0      0.0 
 

the key is('0', 'MZE54.H')
         Total  Percent
qt_360      0      0.0
vt_360      0      0.0 
 

the key is('0', 'SITE_03')
         Total  Percent
qt_360      0      0.0
vt_360      0      0.0 
 

the key is('0', 'SITE_05')
         Total   Percent
qt_360      1  0.000694
vt_360      1  0.000694 
 

the key is('0', 'SITE_08')
         Total   Percent
qt_360      1  0.000694
vt_360      1  0.000694 
 

the key is('1', 'MZE54.3')
         Total   Percent
qt_360      1  0.000694
vt_360      1  0.000694 
 

the key is('1', 'MZE54.4')
         Total  Percent
qt_360      0      0.0
vt_360      0      0.0 
 

the key is('1', 'MZE54.H')
         Total  Percent
qt_360      0      0.0
vt_360      0      0.0 
 

the key is('1', 'MZE54.K')
         Total  Percent
qt_360      0      0.0
vt_360      0      0.0 
 

the key is('1', 'SITE_01')
         Total   Percent
qt_360      7  0.004858
vt_360

In [132]:
for key in  DataGp.keys(): 
         DataGp[key] = DataGp[key].assign(Missing_percentage_qt=(DataGp[key]['qt_360'].isnull().sum()/DataGp[key]['qt_360'].isnull().count()))   
         DataGp[key] = DataGp[key].assign(missing_percentage_vt=(DataGp[key]['vt_360'].isnull().sum()/DataGp[key]['vt_360'].isnull().count()))
        #df_1 = df_1.assign(InterpolateLinear_qt=df_1.qt_360.interpolate(method='linear'))
        
DataGp

{('0',
  'MZE54.3'):      Macrosection    sens code_centrale code_canal                      date  \
 0             I11  SENS_1       MZE54.3          0 2017-05-01 00:00:00+02:00   
 1             I11  SENS_1       MZE54.3          0 2017-05-01 00:06:00+02:00   
 2             I11  SENS_1       MZE54.3          0 2017-05-01 00:12:00+02:00   
 3             I11  SENS_1       MZE54.3          0 2017-05-01 00:18:00+02:00   
 4             I11  SENS_1       MZE54.3          0 2017-05-01 00:24:00+02:00   
 5             I11  SENS_1       MZE54.3          0 2017-05-01 00:30:00+02:00   
 6             I11  SENS_1       MZE54.3          0 2017-05-01 00:36:00+02:00   
 7             I11  SENS_1       MZE54.3          0 2017-05-01 00:42:00+02:00   
 8             I11  SENS_1       MZE54.3          0 2017-05-01 00:48:00+02:00   
 9             I11  SENS_1       MZE54.3          0 2017-05-01 00:54:00+02:00   
 10            I11  SENS_1       MZE54.3          0 2017-05-01 01:00:00+02:00   
 11     

In [129]:
df1 = DataFrameDict.describe(percentiles=[.50,.25,.75 ,.10, .90])
# adding columns for skew and kurtosis  
#df1 = df1.assign(skew_vt=df1['vt_360'].skew(axis = 1,skipna = True))
#df1 = df1.assign(kurt_vt=df1['vt_360'].kurt(axis = 1,skipna = True))

#df1 = df1.drop(columns="Skewness_qt")
#df1 = df1.drop(columns="kurtosis_qt")
#df1 = df1.drop(columns="Skewness_vt")
#df1 = df1.drop(columns="kurtosis_vt")
df1

qt_360                                            \
                           count        mean         std  min   10%    25%   
code_canal code_centrale                                                     
0          MZE54.3        1441.0  131.997224  101.243642  3.0  19.0  35.00   
           MZE54.H        1441.0  131.937543  104.571196  2.0  18.0  32.00   
           SITE_03        1441.0   74.525330   42.641524  2.0  18.0  30.00   
           SITE_05        1440.0   81.536111   50.428037  1.0  18.0  30.75   
           SITE_08        1440.0   85.266667   53.655696  2.0  18.0  30.00   
1          MZE54.3        1440.0  138.448611  104.295013  1.0  17.9  35.75   
           MZE54.4        1441.0  114.712006   91.762312  1.0  14.0  29.00   
           MZE54.H        1441.0  114.734906   84.507736  1.0  16.0  31.00   
           MZE54.K        1441.0  115.067314   91.117928  1.0  14.0  28.00   
           SITE_01        1434.0   36.077406   22.172867  1.0   6.0  13.00   
           SITE_03        1440.0   80.993750   47.843422  1.0  17.0  32.00   
           SITE_05        1439.0   82.904100   49.321395  1.0  17.0  32.00   
           SITE_08        1393.0   11.485284    6.959445  1.0   3.0   6.00   
           SITE_09        1407.0   24.641791   16.507627  1.0   5.0  10.00   
2          MZE54.H        1115.0    5.084305    3.804712  1.0   1.0   2.00   
           SITE_03        1329.0   62.112114   63.156315  1.0   2.0   7.00   
           SITE_05        1352.0   53.303994   54.068890  1.0   2.0   6.00   
           SITE_08        1279.0   52.719312   55.819967  1.0   2.0   5.00   
3          MZE54.H        1311.0   25.618612   28.699247  1.0   2.0   5.00   
           SITE_01        1228.0   28.249186   27.400821  1.0   1.0   4.00   
           SITE_03        1308.0   62.609327   60.093547  1.0   2.0   7.00   
           SITE_05        1326.0   59.076923   59.223696  1.0   2.0   6.00   
           SITE_08        1439.0   76.037526   52.632917  1.0  10.0  23.00   
           SITE_09        1441.0   66.545455   42.590192  1.0  12.0  25.00   
5          SITE_01        1435.0   60.134495   44.403163  1.0   9.0  18.00   
           SITE_08        1416.0   48.093927   47.870633  1.0   3.0   6.00   
           SITE_09        1314.0   56.412481   54.065307  1.0   2.0   7.00   

                                                      vt_360              \
                            50%    75%    90%    max   count        mean   
code_canal code_centrale                                                   
0          MZE54.3        119.0  203.0  273.0  400.0  1441.0   96.670368   
           MZE54.H        119.0  202.0  270.0  450.0  1441.0   90.819570   
           SITE_03         81.0  113.0  126.0  158.0  1441.0   94.820191   
           SITE_05         85.0  124.0  143.0  209.0  1440.0   94.586821   
           SITE_08         90.0  132.0  149.1  205.0  1440.0   84.723101   
1          MZE54.3        143.0  211.0  283.0  414.0  1440.0   96.886806   
           MZE54.4        116.0  173.0  213.0  461.0  1441.0  103.768217   
           MZE54.H        117.0  177.0  227.0  358.0  1441.0   98.566273   
           MZE54.K        114.0  176.0  219.0  433.0  1441.0   98.297016   
           SITE_01         42.0   55.0   63.0   85.0  1435.0  101.315459   
           SITE_03         97.0  121.0  139.1  175.0  1440.0   97.071148   
           SITE_05         98.0  126.0  141.0  184.0  1439.0  100.883613   
           SITE_08         10.0   17.0   21.0   49.0     0.0         NaN   
           SITE_09         21.0   40.0   47.0   70.0  1406.0   79.502892   
2          MZE54.H          4.0    7.0   10.0   20.0  1114.0   59.887792   
           SITE_03         47.0   93.0  158.2  281.0  1306.0  107.896635   
           SITE_05         40.0   80.0  133.9  228.0  1344.0  108.659105   
           SITE_08         41.0   77.0  131.0  246.0  1259.0   93.042972   
3          MZE54.H         18.0   35.0   50.0  171.0  1309.0   53.873950   
           SI

In [130]:
#df1.insert(loc=10, column='Skewness_qt', value=DataFrameDict['qt_360'].skew())
#df1.insert(loc=11, column='kurtosis_qt', value=DataFrameDict['qt_360'].kurt())

#df1.insert(loc=21, column='Skewness_vt', value=DataFrameDict['vt_360'].skew())
#df1.insert(loc=23, column='kurtosis_vt', value=DataFrameDict['vt_360'].kurt())

#df1.insert(loc=12, column='Qt missing data %', value=  DataFrameDict['qt_360'].isnull().sum()/DataFrameDict['qt_360'].isnull().count())
#df1.insert(loc=25, column='Vt missing data %', value=  DataFrameDict['vt_360'].isnull().sum()/DataFrameDict['vt_360'].isnull().count())


In [121]:
display(df1)

qt_360                                            \
                           count        mean         std  min   10%    25%   
code_canal code_centrale                                                     
0          MZE54.3        1441.0  131.997224  101.243642  3.0  19.0  35.00   
           MZE54.H        1441.0  131.937543  104.571196  2.0  18.0  32.00   
           SITE_03        1441.0   74.525330   42.641524  2.0  18.0  30.00   
           SITE_05        1440.0   81.536111   50.428037  1.0  18.0  30.75   
           SITE_08        1440.0   85.266667   53.655696  2.0  18.0  30.00   
1          MZE54.3        1440.0  138.448611  104.295013  1.0  17.9  35.75   
           MZE54.4        1441.0  114.712006   91.762312  1.0  14.0  29.00   
           MZE54.H        1441.0  114.734906   84.507736  1.0  16.0  31.00   
           MZE54.K        1441.0  115.067314   91.117928  1.0  14.0  28.00   
           SITE_01        1434.0   36.077406   22.172867  1.0   6.0  13.00   
           SITE_03        1440.0   80.993750   47.843422  1.0  17.0  32.00   
           SITE_05        1439.0   82.904100   49.321395  1.0  17.0  32.00   
           SITE_08        1393.0   11.485284    6.959445  1.0   3.0   6.00   
           SITE_09        1407.0   24.641791   16.507627  1.0   5.0  10.00   
2          MZE54.H        1115.0    5.084305    3.804712  1.0   1.0   2.00   
           SITE_03        1329.0   62.112114   63.156315  1.0   2.0   7.00   
           SITE_05        1352.0   53.303994   54.068890  1.0   2.0   6.00   
           SITE_08        1279.0   52.719312   55.819967  1.0   2.0   5.00   
3          MZE54.H        1311.0   25.618612   28.699247  1.0   2.0   5.00   
           SITE_01        1228.0   28.249186   27.400821  1.0   1.0   4.00   
           SITE_03        1308.0   62.609327   60.093547  1.0   2.0   7.00   
           SITE_05        1326.0   59.076923   59.223696  1.0   2.0   6.00   
           SITE_08        1439.0   76.037526   52.632917  1.0  10.0  23.00   
           SITE_09        1441.0   66.545455   42.590192  1.0  12.0  25.00   
5          SITE_01        1435.0   60.134495   44.403163  1.0   9.0  18.00   
           SITE_08        1416.0   48.093927   47.870633  1.0   3.0   6.00   
           SITE_09        1314.0   56.412481   54.065307  1.0   2.0   7.00   

                                                     Skewness_qt  vt_360  \
                            50%    75%    90%    max               count   
code_canal code_centrale                                                   
0          MZE54.3        119.0  203.0  273.0  400.0    0.527611  1441.0   
           MZE54.H        119.0  202.0  270.0  450.0    0.664985  1441.0   
           SITE_03         81.0  113.0  126.0  158.0   -0.125024  1441.0   
           SITE_05         85.0  124.0  143.0  209.0    0.075056  1440.0   
           SITE_08         90.0  132.0  149.1  205.0    0.039646  1440.0   
1          MZE54.3        143.0  211.0  283.0  414.0    0.423421  1440.0   
           MZE54.4        116.0  173.0  213.0  461.0    0.918163  1441.0   
           MZE54.H        117.0  177.0  227.0  358.0    0.358238  1441.0   
           MZE54.K        114.0  176.0  219.0  433.0    0.740694  1441.0   
           SITE_01         42.0   55.0   63.0   85.0   -0.090922  1435.0   
           SITE_03         97.0  121.0  139.1  175.0   -0.103056  1440.0   
           SITE_05         98.0  126.0  141.0  184.0   -0.117814  1439.0   
           SITE_08         10.0   17.0   21.0   49.0    0.532424     0.0   
           SITE_09         21.0   40.0   47.0   70.0    0.368963  1406.0   
2          MZE54.H          4.0    7.0   10.0   20.0    1.269145  1114.0   
           SITE_03         47.0   93.0  158.2  281.0    1.104183  1306.0   
           SITE_05         40.0   80.0  133.9  228.0    1.068353  1344.0   
           SITE_08         41.0   77.0  131.0  246.0    1.334367  1259.0   
3          MZE54.H         18.0   35.0   50.0  171.0    2.418658  1309.0   
           SI

### outliers detection

(506, 13)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x=)